In [23]:
from init import *

In [24]:
import plotly.express as px
def linspace(min:float,max:float,n:int):
    assert n>=2 and min!=max
    step = (max-min)/n
    return [min] + [min+step*i for i in range(1,n-2)] + [max]

def PlotHistogram1D (h):
    bins = h.bins
    xmin = h.x.min
    xmax = h.x.max
    nx   = h.x.nbins
    assert nx==len(bins)
    dx = (xmax-xmin)/nx
    return px.histogram(x=[xmin+dx*i for i in range(nx)],y=bins,range_x=[xmin,xmax],nbins=nx)

def PlotHistogram2D (h):
    bins = h.bins
    xmin = h.x.min
    xmax = h.x.max
    nx   = h.x.nbins
    ymin = h.y.min
    ymax = h.y.max
    ny   = h.y.nbins
    assert nx*ny==len(bins)
    dx = (xmax-xmin)/nx
    dy = (ymax-ymin)/ny
    z = [[bins[iy+ix*ny] for ix in range(nx)] for iy in range(ny)]
    return px.imshow(z,aspect='equal')
    
def PlotHistogram (h):
    if getattr(h,'y',None):
        return PlotHistogram2D(h)
    else:
        return PlotHistogram1D(h)

# Example with correlated processes

In [25]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 1000
model.NumPaths = 100000
model.MemoryLimitKB = 200
correlation = -0.5
W1_number = 0
W2_number = 1

model.Add(sdk.IndependentGaussian([W1_number]))
model.Add(sdk.CorrelatedGaussian(correlation,W1_number,W2_number))

W1 = sdk.BrownianMotion(0,0,1)
model.Add(W1) # start, drift, diffusion
assert W1.GetStateNumber()==W1_number

W2 = sdk.BrownianMotion(0,0,1)
model.Add(W2) # start, drift, diffusion
assert W2.GetStateNumber()==W2_number

model.TimeSteps = 1000
for t in [1,2,5]:
    model.evaluations.append (
        sdk.EvaluationPoint(state=0,time=t)
            .Add(sdk.Histogram(
                x = sdk.HistogramAxis(W1.GetStateNumber(),100,-5,5),
            ))
            .Add(sdk.Histogram(
                x = sdk.HistogramAxis(W2.GetStateNumber(),100,-5,5),
            ))
            .Add(sdk.Histogram(
                x = sdk.HistogramAxis(W1.GetStateNumber(),100,-5,5),
                y = sdk.HistogramAxis(W2.GetStateNumber(),100,-5,5)
            ))
    )

er = run(model)
er.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,200,100000,-0.003321,0.003161,0.999679,-0.008095
1,BrownianMotion,,1,1.0,200,100000,-0.001158,0.003161,0.999645,0.004233
2,BrownianMotion,,0,2.0,400,100000,-0.002370,0.004480,1.416610,0.002323
3,BrownianMotion,,1,2.0,400,100000,-0.002938,0.004484,1.417838,0.003695
4,BrownianMotion,,0,5.0,1000,100000,0.002033,0.007069,2.235357,0.001516
5,BrownianMotion,,1,5.0,1000,100000,-0.008221,0.007053,2.230256,0.002720


In [26]:
PlotHistogram(er.histograms[0])

In [27]:
PlotHistogram(er.histograms[1])

In [28]:
PlotHistogram(er.histograms[2])

In [29]:
PlotHistogram(er.histograms[3])

In [30]:
PlotHistogram(er.histograms[4])

In [31]:
PlotHistogram(er.histograms[5])

In [32]:
PlotHistogram(er.histograms[6])

In [33]:
PlotHistogram(er.histograms[7])

In [34]:
PlotHistogram(er.histograms[8])